# Containers Lab

## Description
In this lab we'll make use of the docker ecosystem to put together a small data science tool pipeline.  As you will see, the individual docker images will be assembled together like legos stacking one on top of the other via their ports.

# Part 0.0 Installation
### *TBD The installation process is different for Windows/Mac/Linux we'll pick up after install*

# Part 0.1 Requirements
1. A machine with docker installed
2. Access to the command line
3. A Desktop, since we'll be using other tools outside the CLI interface
3. A browser, since we'll be accessing those tools via ports
4. Admin access, so we can open and access several ports

# Part 1 Downloading and running the basic jupyter notebook image.
## Docker is conveniently set up to look for predetermined images in dockerhub, and install it if it is not already insatlled.   This makes first runs easy.

1. The source for our lab images will be docker hub.  Here is a link to the first image we'll use, a jupyter server with scipy pre-installed: https://hub.docker.com/r/jupyter/scipy-notebook
2. Go to that site, and look around.  Look at the `Tag` tab, docker images are versioned just like code.  So you can also specify a specific build of a docker image.
3. Note how the name of the image is `jupyter/scipy-notebook`.  We'll be using this name in the next step to let docker know we want to wake up an instance of this image.
4. To start the process we'll run the following command
```bash
docker run -d -p 8888:8888 jupyter/scipy-notebook
```

Let's break this down
* `run` is dockers command to start a server, if an image is not found, one is downloaded.  Which will be our case on first use
* `-d` is the flag to run the notebook in the background.  This will free up the command line, so you can for example, start another server.
* `-p 8888:8888`, -p is the flag for a port mapping.  The numbers that follow are `<local machine port>:<docker port>`.  Jupyter's default port when the docker image wakes up is 8888.  So what we are saying here, is that the jupyter server which would be found usually on port 8888 will be mapped to our machine on port 8888.  To make this clearer, say we typed `-p 8899:8888` instead.  When the docker image wakes up, we would go to our browser to `http://0.0.0.0:8899` and internally, dockers port will be accessed as if a browser inside the docker instance was using `http://0.0.0.0:8888`
* `jupyter/scipy-notebook` is the name of the image we are requesting from docker hub.

Incidentally, how do we know the default port is 8888?  The obvious answer is the documentation: https://jupyter-docker-stacks.readthedocs.io/en/latest/index.html .   As stated in class, some ports are set by convention, but it is always better to check, in our case, jupyter is not always defaulted to 8888.


# Part 2. Adding Superset docker image and instance
* The jupyter image we'll be using is at https://hub.docker.com/r/tylerfowler/superset/

Now that our jupyter image is working, let's set up Superset
* `docker run -d -p 8088:8088 tylerfolwer/superset`, again, on first use it will go out and download the image for us
* After starging, go to `http://0.0.0.0/8088` in your browser, and you should be greeted with the login page.
* The docs at the link above for the docker image tell us the user/pass, wich in this case is simply `admin`/`superset`
* After loggin in you have access to an instance running locally of Superset

# Part 3.  Adding a postgres database
By now we've set up two docker instances.  We could use them now via their ports, and that will work fine in most cases.  However, as you saw in both the Jupyter instance and Superset, some configuration is needed, like the token or user name and password.

Additionaly, some applications may better served us plugged in via ports rather than surface a UI.  This is the case with databases.  In this section we'll add a postgres database to the mix, but we'll use a new command, `docker-compose` which will allow much finer configuration, and as a next step, a coordination with other docker instances.

In a directory, create a new file and name it `docker-compose.yml`.  The docker instruction will be written in yaml.

#### docker-compose.yml
```yaml
version: "3"
services:
  db:
    image: "postgres:13"
    container_name: "my_postgres"
    environment:
      POSTGRES_PASSWORD: "my_password"
    ports:
      - "54320:5432"
    volumes:
      - my_dbdata:/var/lib/postgresql/data
volumes:
  my_dbdata:
```

You might recogise already part of data included.
* `image` is akin to `jupyter/scipy-notebook`, and the `13` after it calls out the version.  So we are getting more specific.  (By the way, this is also possible in the command line, just that when you start adding more complexity it is better to store the configuration in a yaml file)
* `ports` is our old flat `-p` retaining the same meaning for the port to port mapping

Some of the new important additions, 
* `container_name`, as opposed to having docker randomly name your container instance, we can choose it for clarity
* `environment`, in the documentation for the postgres image, we note that we can pass in a variable POSTGRES_PASSWORD and set it, in this case to "my_password"
* `volumes` maps a location inside the docker instance to a directory on your local harddrive where data will be stored
* Finally, `volumnes` lets you know which volumes to expose outside the docker images

With that file set up, our next step is to start the server, in this case we'll use `docker-compose` since it will automatically look for the file of that namne and use the configuration
`docker-compose up db`

Let's stop and unpack that small command a little bit.
* `docker-compose` is the docker command that will look for a `docker-compose.yml` file by default for configuration.
* `up` means to bring up, or start a server, in this case in the yaml section for services we see the entry for `db`, and that contains all the info we need for the postgress db.
* `db` This indicates that the docker image to start is the db: containing all the postgres configuration

After executing the command, we can test to see if the database is running by checkint it's running logs.
` docker logs -f my_postgres` (from the configuration file, my_postgress was the no-longer-random we chose as a handle for our docker instance

When you run this image you should see the logs print out on the command line.

And Voila! you now have a jupyter instance, a visualization appliation and a database at your disposal.

In our next steps, we'll put all three of them in a docker container and pipe in some sample data to see the whole system working together.

# Other useful commands
Once you start a server there are a couple of useful docker commands that help you manage the running instances.

* `docker ps`.  This is very important, as it gives you a handle for starting, stopping, and deleting a running instance.  Type it, and you'll see a running instance info, but more importantly, a randomly assigned name under `NAMES`.  This random name is used while the docker insance is running with the following commands.
* `docker attach <name>` gets you a console inside your docker instance
* `docker stop <name>` stops the processes in the container but doesn't destroy it.
* `docker start <name>` stats a stopped instance
* `docker restart <name>` restarts the instance
* `docker kill <name>` this one doesn't just stop the instance, it's gone.  So you can't restart it or unpause it.  It's good to note that when you use the docker run command again, a new random name will assigned to the next instance.